# Importing required packages

In [0]:
#pip install openpyxl
import pandas as pd


# Reading Train and Test data

In [318]:
Data_Train = pd.read_excel("Data_Train.xlsx")
Data_Test = pd.read_excel("Data_Test.xlsx")
print("Number of records in Train",len(Data_Train))
print("Number of records in Test",len(Data_Test))

Number of records in Train 11094
Number of records in Test 2774


In [319]:
Data_Train.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [320]:
Data_Train.describe()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
count,11094,11094,11094,11094,11094,11094,11094,11094,11094
unique,7480,35,2179,26,18,33,1103,761,7
top,ID_7184,"Mico Layout, Stage 2, BTM Layout,Bangalore",North Indian,₹200,₹50,-,-,-,30 minutes
freq,22,947,850,3241,10118,1191,2074,2312,7406


# Preparing the data

In [0]:
X = Data_Train.copy() #Creating a new copy of Train data

In [0]:
#X.dropna(axis=0,subset=['Delivery_Time'],inplace=True)  #Dropping any rows with the predicting feature empty/NaN
#y= X.Delivery_Time  #Setting the predicting feature
#X.drop(['Delivery_Time'],axis=1,inplace=True) #Removing the Predicting feature from X

In [0]:
#X.loc[X.Minimum_Order.isin(["₹50"])]

In [324]:
X.columns

Index(['Restaurant', 'Location', 'Cuisines', 'Average_Cost', 'Minimum_Order',
       'Rating', 'Votes', 'Reviews', 'Delivery_Time'],
      dtype='object')

In [325]:
#features = ["Location","Average_Cost","Minimum_Order","Rating",	"Votes", "Reviews","Delivery_Time"]
features = ["Location","Rating",	"Votes", "Reviews","Delivery_Time"]
X_f = X[features]
X_f.dropna(axis=0,subset=features,inplace=True)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [326]:
X_f.head()

,Location,Rating,Votes,Reviews,Delivery_Time
0,"FTI College, Law College Road, Pune",3.5,12,4,30 minutes
1,"Sector 3, Marathalli",3.5,11,4,30 minutes
2,Mumbai Central,3.6,99,30,65 minutes
3,"Sector 1, Noida",3.7,176,95,30 minutes
4,"Rmz Centennial, I Gate, Whitefield",3.2,521,235,65 minutes


In [327]:
X_f.describe()

,Location,Rating,Votes,Reviews,Delivery_Time
count,11094,11094,11094,11094,11094
unique,35,33,1103,761,7
top,"Mico Layout, Stage 2, BTM Layout,Bangalore",-,-,-,30 minutes
freq,947,1191,2074,2312,7406


In [0]:
locations = X.Location

In [0]:
locations = locations.astype("str")

In [0]:
l = locations[0]

In [331]:
print(l.rsplit(sep=", "))
city = l.rsplit(sep=", ")[-1]
print(city)

['FTI College', 'Law College Road', 'Pune']
Pune


In [0]:
def getCity(X_Locations):
  X_Locations = X_Locations.astype("str")
  city = []
  for l in X_Locations:
    city.append(l.rsplit(sep = ", ")[-1])
  return city

In [333]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

city = getCity(X_f.Location)
#le.fit(city)
X_f["City"] = le.fit_transform(city)
X_f = X_f.drop("Location",axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [334]:
X_f.head()

,Rating,Votes,Reviews,Delivery_Time,City
0,3.5,12,4,30 minutes,17
1,3.5,11,4,30 minutes,12
2,3.6,99,30,65 minutes,15
3,3.7,176,95,30 minutes,16
4,3.2,521,235,65 minutes,21


In [335]:
X_f.Rating.replace(["NEW","-","Opening Soon",'Temporarily Closed'],2.5,inplace=True)
X_f.Votes.replace(["-"],0,inplace=True)
X_f.Reviews.replace(["-"],0,inplace=True)
'''
for col in X_f.index:
  s = str(X_f.Average_Cost[col])
  if s.isalpha():
    print("Average cost has a non numeric value ",s)
    X_f.drop(index=col,axis=0,inplace=True)
    continue
  
  X_f.Average_Cost[col]=X_f.Average_Cost[col].replace("₹","")
  X_f.Average_Cost[col]=X_f.Average_Cost[col].replace(",","")
  X_f.Minimum_Order[col]=X_f.Minimum_Order[col].replace("₹","")
  X_f.Minimum_Order[col]=X_f.Minimum_Order[col].replace(",","")
'''


'\nfor col in X_f.index:\n  s = str(X_f.Average_Cost[col])\n  if s.isalpha():\n    print("Average cost has a non numeric value ",s)\n    X_f.drop(index=col,axis=0,inplace=True)\n    continue\n  \n  X_f.Average_Cost[col]=X_f.Average_Cost[col].replace("₹","")\n  X_f.Average_Cost[col]=X_f.Average_Cost[col].replace(",","")\n  X_f.Minimum_Order[col]=X_f.Minimum_Order[col].replace("₹","")\n  X_f.Minimum_Order[col]=X_f.Minimum_Order[col].replace(",","")\n'

In [336]:
X_f

,Rating,Votes,Reviews,Delivery_Time,City
0,3.5,12,4,30 minutes,17
1,3.5,11,4,30 minutes,12
2,3.6,99,30,65 minutes,15
3,3.7,176,95,30 minutes,16
4,3.2,521,235,65 minutes,21
...,...,...,...,...,...
11089,4.2,326,189,30 minutes,5
11090,3.6,36,16,30 minutes,16
11091,3.5,45,18,30 minutes,11
11092,3.1,24,9,30 minutes,12


In [337]:
y_f = X_f.Delivery_Time
X_f.drop(["Delivery_Time"],axis=1,inplace=True)
X_f.astype("float32").dtypes

Rating     float32
Votes      float32
Reviews    float32
City       float32
dtype: object

In [338]:
X_f.shape

(11094, 4)

In [339]:
y_f

0        30 minutes
1        30 minutes
2        65 minutes
3        30 minutes
4        65 minutes
            ...    
11089    30 minutes
11090    30 minutes
11091    30 minutes
11092    30 minutes
11093    30 minutes
Name: Delivery_Time, Length: 11094, dtype: object

In [340]:
print(y_rating[0].replace("minutes",""))

30 


In [341]:
for col in y_f.index:
  y_f[col]=y_f[col].replace("minutes","")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [342]:
y_f

0        30 
1        30 
2        65 
3        30 
4        65 
        ... 
11089    30 
11090    30 
11091    30 
11092    30 
11093    30 
Name: Delivery_Time, Length: 11094, dtype: object

In [343]:
y_f.astype("int32").dtypes

dtype('int32')

In [0]:
from sklearn.model_selection import train_test_split
X_train,X_valid,y_train,y_valid = train_test_split(X_f,y_f)

# Modelling

In [345]:
from sklearn.ensemble import RandomForestRegressor
model_1 = RandomForestRegressor(n_estimators=500,random_state=1)
model_1.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=500,
                      n_jobs=None, oob_score=False, random_state=1, verbose=0,
                      warm_start=False)

In [0]:
pred = model_1.predict(X_valid)

In [0]:
from sklearn.metrics import mean_absolute_error
score = mean_absolute_error(pred,y_valid)

In [348]:
score

0.0005912040374909867

In [0]:
def getScores(n_est,X_train,y_train,X_valid,y_valid):
  scores = {}
  for k in n_est:
    model = RandomForestRegressor(n_estimators=k,random_state=1)
    model.fit(X_train,y_train)
    preds = model.predict(X_valid)
    score = mean_absolute_error(preds,y_valid)
    scores[k]=score
  return scores

In [0]:
n_est = [50,100,150,200,500,1000,1500,2000]
scores = getScores(n_est, X_train, y_train, X_valid, y_valid)

In [351]:
scores

{50: 0.0003604902667627974,
 100: 0.0003604902667627974,
 150: 0.0005046863734679165,
 200: 0.0005046863734679171,
 500: 0.0005912040374909867,
 1000: 0.000544340302811824,
 1500: 0.0004998798365777457,
 2000: 0.0004983777937995547}

In [352]:
0.0002162941600576783 < 0.00021629416005767896

True

In [353]:
X_f.columns

Index(['Rating', 'Votes', 'Reviews', 'City', 'Delivery_Time'], dtype='object')

In [354]:
if "Delivery_Time" in X_f.columns:
  print("hi")

hi


In [355]:
model_test = RandomForestRegressor(n_estimators=200,random_state=1)  #choosing the best n_est
if "Delivery_Time" in X_f.columns:
  X_f.drop(["Delivery_Time"],axis=1,inplace=True)
model_test.fit(X_f,y_f) #fitting on the complete Training data
X_test = Data_Test.copy()
if "Delivery_Time" in features:
  features.remove("Delivery_Time")
X_test = X_test[features]
city = getCity(X_test.Location)
#le.fit(city)
X_test["City"] = le.fit_transform(city)
X_test = X_test.drop("Location",axis=1)
X_test.Rating.replace(["NEW","-","Opening Soon",'Temporarily Closed'],2.5,inplace=True)
X_test.Votes.replace(["-"],0,inplace=True)
X_test.Reviews.replace(["-"],0,inplace=True)
'''
for col in X_test.index:
  s = str(X_test.Average_Cost[col])
  if s.isalpha():
    print("Average cost has a non numeric value ",s)
    X_test.drop(index=col,axis=0,inplace=True)
    continue
  
  X_test.Average_Cost[col]=X_test.Average_Cost[col].replace("₹","")
  X_test.Average_Cost[col]=X_test.Average_Cost[col].replace(",","")
  X_test.Minimum_Order[col]=X_test.Minimum_Order[col].replace("₹","")
  X_test.Minimum_Order[col]=X_test.Minimum_Order[col].replace(",","")
'''
X_test.astype("float32").dtypes

Rating     float32
Votes      float32
Reviews    float32
City       float32
dtype: object

In [356]:
X_test.head()

,Rating,Votes,Reviews,City
0,4.2,361,225,0
1,2.5,0,0,0
2,3.6,36,16,16
3,3.6,66,33,2
4,2.9,38,14,16


In [0]:
#import numpy as np
#X_test = np.array(X_test)
#X_test = X_test.reshape(-1,1)

In [358]:
X_test.shape

(2774, 4)

In [0]:
predictions_test = model_test.predict(X_test)

# Converting time multiples of 5

In [0]:
import math
pred_minutes = []
for p in predictions_test:
    time = math.ceil(p)
    d = time // 5
    time = d * 5
    pred_minutes.append(str(time) + " minutes")

In [0]:
#pred_minutes

# Creating a submission excel file

In [0]:
output = pd.DataFrame({'Delivery_Time':pred_minutes})

In [0]:
output.to_excel("Submission_6.xlsx",index=False)